# Discretize trajectory  

Loads a trajectory of two particles/molecules computed with the benchmark model (positions and orientations) and discretizes it into a discrete trajectory depending on their relativ position and orientation. This is just a draft of the scripts used in the scripts folder.

In [ ]:
import pickle
import numpy as np
import msmrd2
import msmrd2.tools.trajectoryTools as trajectoryTools
import msmrd2.tools.analysis as analysisTools
import timeit

In [ ]:
# Load parameters from parameters file
parentDirectory = '../../data/patchyDimer/benchmark/'
parameterDictionary = analysisTools.readParameters(parentDirectory + "parameters")
# Parameters for loading continuous trajectories from files (from original simulation)
nfiles = 5 #parameterDictionary['numFiles']
dt = parameterDictionary['dt'] 
stride = parameterDictionary['stride']
totalTimeSteps = parameterDictionary['timesteps'] 
boxsize = parameterDictionary['boxsize']
boundaryType = parameterDictionary['boundaryType']
parameterDictionary

In [ ]:
# Calculated parameters
#dtEffective = dt*stride # needed to obtain rate dictionary
effectivetimeSteps = int(totalTimeSteps/stride)
fnamebase = parentDirectory + 'simDimer_'
#fnamebase = parentDirectory + 'simPatchyProtein_'

In [ ]:
# Setup for discrete trajectory (in this case patchy dimer)
numParticles = parameterDictionary['numParticles']
bufferSize = effectivetimeSteps

# Set trajectory discretizator
radialLowerBound = 1.25
radialUpperBound = 2.25
#discretizator = msmrd2.trajectories.patchyDimer(numParticles, bufferSize)
discretizator = msmrd2.trajectories.patchyDimer2(numParticles, bufferSize, radialLowerBound, radialUpperBound)
#discretizator = msmrd2.trajectories.patchyProtein(numParticles, bufferSize, radialLowerBound, radialUpperBound)
#discretizator = msmrd2.trajectories.patchyProtein2(numParticles, bufferSize, radialLowerBound, radialUpperBound)

# Set boundary (important for discretizer)
boxBoundary = msmrd2.box(boxsize, boxsize, boxsize, boundaryType)
discretizator.setBoundary(boxBoundary)

## Discretization (fast approach)

In [ ]:
# Load H5 files and generates discrete trajectories at once directly on c++.
dtrajs = []
for i in range(nfiles):
    filename = fnamebase + str(i).zfill(4) + '.h5'
    dtraj = discretizator.discretizeTrajectoryH5(filename)
    dtrajs.append(dtraj)
    if i%10 == 0:
        print("File ", i+1, " of ", nfiles, " done.", end="\r")
print("Done")

In [ ]:
# Write discrete trajectory to xyz file
for i, dtraj in enumerate(dtrajs):
    datafile  = open(fnamebase + str(i).zfill(4) + '_discrete.xyz', 'w')
    for j in range(len(dtraj)):
        datafile.write(str(dtraj[j]) + '\n')
    datafile.close()
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

Here you are done with the discretization. Below are additional ways to do the discretization and some tests. 

## Alternative discretization approach (older approach, slightly slower)

Loading first H5 file in python and the discretizing. Could be useful in some cases.

In [ ]:
# Load trajectories
trajs = []
for i in range(nfiles):
    traj = trajectoryTools.loadTrajectory(fnamebase, i)
    trajs.append(traj)    
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

In [ ]:
# Generate discrete trajectories using msmrd2.trajectories.discrete
dtrajs = []
for i in range(nfiles):
    currentTraj = trajs[i]
    dtraj = discretizator.discretizeTrajectory(currentTraj)
    dtrajs.append(dtraj)
    if i%20 == 0:
        print("File ", i+1, " of ", nfiles, " done.", end="\r")
print("Done")

In [ ]:
# Write discrete trajectory to xyz file
for i, dtraj in enumerate(dtrajs):
    datafile  = open(fnamebase + str(i).zfill(4) + '_discrete_python_debug.xyz', 'w')
    for j in range(len(dtraj)):
        datafile.write(str(dtraj[j]) + '\n')
    datafile.close()
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

## Alternative discretization approach (older approach) 

Uses batches, so it is better for systems with small memory and for debugging)

In [ ]:
# Load trajectories
trajs = []
fileNum1 = 258
fileNum2 = 259
for i in range(fileNum1, fileNum2):
    traj = trajectoryTools.loadTrajectory(fnamebase, i)
    trajs.append(traj)    
    print("File ", i+1, " of ", fileNum2, " done.", end="\r")

In [ ]:
# Generate discrete trajectories using msmrd2.trajectories.discrete
dtrajs = []
for i in range(fileNum2 - fileNum1):
    currentTraj = trajs[i]
    dtraj = discretizator.discretizeTrajectory(currentTraj)
    dtrajs.append(dtraj)
    if i%20 == 0:
        print("File ", i+1, " of ", fileNum2 - fileNum1, " done.", end="\r")
print("Done")

In [ ]:
# Choose a specific part to debug
particles = trajs[0][2*200156:2*200157]

In [ ]:
# Create dummy particles to evaluate state
part1 = msmrd2.particle(0,0, particles[0][1:4], particles[0][4:8])
part2 = msmrd2.particle(0,0, particles[1][1:4], particles[1][4:8])

In [ ]:
# Evaluate current state
discretizator.sampleDiscreteState(part1,part2)

In [ ]:
# Write discrete trajectory to xyz file
for i, dtraj in enumerate(dtrajs):
    datafile  = open(fnamebase + str(fileNum1 + i).zfill(4) + '_discrete_python_debug.xyz', 'w')
    for j in range(len(dtraj)):
        datafile.write(str(dtraj[j]) + '\n')
    datafile.close()
    print("File ", i+1, " of ", fileNum2 - fileNum1, " done.", end="\r")

## Timeit tests
Showing it is faster the first way

In [ ]:
mysetup = '''
import numpy as np
import msmrd2
import msmrd2.tools.trajectoryTools as trajectoryTools

nfiles = 10
dt = 0.00001 # from original simulation
stride = 25  # stride of sampling (every 25th dt)
totalTimeSteps = 4000000 #2000000
boxsize = 4
boundaryType = 'periodic'

# Calculated parameters
dtEffective = dt*stride # needed to obtain rate dictionary
effectivetimeSteps = int(totalTimeSteps/25)
fnamebase = '../data/dimer/simDimer_t' + "{:.2E}".format(totalTimeSteps) + "_s{:d}".format(stride) + "_"

# Setup for discrete trajectory (in this case patchy dimer)
numParticles = 2
bufferSize = effectivetimeSteps
discretizator = msmrd2.trajectories.patchyDimer(numParticles, effectivetimeSteps)
# Set boundary (important for discretizer)
boxBoundary = msmrd2.box(boxsize, boxsize, boxsize, boundaryType)
discretizator.setBoundary(boxBoundary)
'''

In [ ]:
mycode1 = '''
trajs = []
for i in range(nfiles):
    traj = trajectoryTools.loadTrajectory(fnamebase, i)
    trajs.append(traj)    

# Generate discrete trajectories using msmrd2.trajectories.discrete
dtrajs = []
for i in range(nfiles):
    currentTraj = trajs[i]
    dtraj = discretizator.discretizeTrajectory(currentTraj)
    dtrajs.append(dtraj)
'''

In [ ]:
time = timeit.timeit(setup = mysetup, stmt = mycode1, number = 1)
print(time)

In [ ]:
mycode2 = '''
dtrajs = []
for i in range(nfiles):
    filename = fnamebase + str(i).zfill(4) + '.h5'
    dtraj = discretizator.discretizeTrajectoryH5(filename)
    dtrajs.append(dtraj)
'''

In [ ]:
time2 = timeit.timeit(setup = mysetup, stmt = mycode2, number = 1)
print(time2)